In [8]:
import chi
import os

#Config with your project and site
chi.use_site("CHI@UC")  # Optional, defaults to 'CHI@UC'
chi.set("project_name", "CHI-210833")  # Replace with your project name

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'Subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Insert keypair name
key_name = 'msherman-laptop'  # Change to your keypair name

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


In [6]:
from chi import lease
reservations = []

#Image to use on your servers
image_name='CC-Ubuntu20.04-CUDA11'
node_count = 2
node_type = "gpu_rtx_6000"

lease.add_node_reservation(reservations, count=node_count, node_type=node_type)
lease.add_fip_reservation(reservations, count=node_count)

lease.add_network_reservation(reservations, network_name)

start_date, end_date = lease.lease_duration(days=1)

leased_resources = lease.create_lease(lease_name, reservations, start_date=start_date, end_date=end_date)

In [7]:
from chi import network
lease_id = leased_resources["id"]
lease.wait_for_active(lease_id)
lease_server_id = lease.get_node_reservation(lease_id)

network_id = lease.get_network_id(network_name)
network.create_subnet(subnet_name, network_id)
network.create_router(router_name, gw_network_name=network.PUBLIC_NETWORK)

router_id = network.get_router_id(router_name)
subnet_id = network.get_subnet_id(subnet_name)
network.add_subnet_to_router(router_id, subnet_id)

{'network_id': 'a7ded07f-e1bb-4a2f-affe-2d29913b9d47',
 'tenant_id': '09f6fb74a74245a6aa1e0d671eb30c95',
 'subnet_id': '08d6a65d-d48f-44f0-93d4-76bf1120438e',
 'subnet_ids': ['08d6a65d-d48f-44f0-93d4-76bf1120438e'],
 'port_id': '3d455e03-8424-4766-8edb-967e0d7d98ea',
 'id': 'b104abbe-35ba-4998-b9d6-ee6e1ed08c8c'}

In [9]:
from chi import server
server_obj = server.create_server("scc-test-server", 
                                  reservation_id = lease_server_id,
                                  network_name = network_name,
                                  key_name = key_name,
                                  image_name=image_name,
                                  count = node_count
                                 )

In [11]:
print(server.wait_for_active(server_obj[0].id).status)
print(server.wait_for_active(server_obj[1].id).status)

ResourceFailure: Server:a23fa0a7-70fc-4d64-a0ad-c5bfa5876b7a transitioned to failure state ERROR

In [ ]:
ip_0 = server.associate_floating_ip(server_obj[0].id)
ip_1 = server.associate_floating_ip(server_obj[1].id)

In [ ]:
server.wait_for_tcp(ip_0, 22)
server.wait_for_tcp(ip_1, 22)
